## Week 5
### Unobservable model errors. Reject inference. Metalearning.

In this jupyter-notebook we will learn how to use reject inference for credit scoring data. Then we will consider two different models trained on not fully known data. We will try to impute missing target values  using predictions of both models with the help of metalearning. Finally, we will build two approximated benefit curves for two different models.

#### Import libraries

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix


from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import f1_score as f1

import matplotlib.colors


sns.set(style="white")

np.random.seed(2020)

#settings for plots
plt.rcParams.update({'font.size': 16,
                     'xtick.labelsize' : 14, 
                     'ytick.labelsize' : 14,
                     'axes.labelsize' : 16,
                     'axes.titlesize' : 20})

In [ ]:
import warnings
warnings.filterwarnings("ignore")

Define some functions that will help us to plot graphs. 

In [ ]:
def benefit_plot(X_train, prob_col = 'prob', leg=' ', linest='-', plot_y_true = True):
    
    if linest == '-':
        plt.figure(figsize=(12,8))
        
    if prob_col == 'prob':
        colors = ['olivedrab','deepskyblue']
        reconstr_type = 'Reject inference'
    else:
        colors = ['salmon', 'deepskyblue']
        reconstr_type = 'Metalearning'
        
    max_val = []
    if plot_y_true:
        cols_to_plot = ['y','y_hat']
    else:
        cols_to_plot = ['y_hat']
    
    
    for it, i in enumerate(cols_to_plot):

        benefit = []    
        c_acceptance_rate = []
        
        y_hat = X_train[i]
        y_ar = X_train[prob_col]
        
        for t in thr:            
            #calculate accaptance rate as amount of non-defaulted clients
            c_acceptance_rate.append((len(y_ar) - np.sum((y_ar > t)*1.)) / len(y_ar))
            curr_y_hat = y_hat[X_train[prob_col] <= t]    
            #calculate the financial effect
            benefit.append(np.sum((1 - curr_y_hat)* e_fp - curr_y_hat * e_fn))
        if i == 'y':
            plt.plot(c_acceptance_rate, benefit, label = 'True benefit for ' + leg,\
                     color = colors[it], linewidth=2, ls = linest)           
        
        else:
            plt.plot(c_acceptance_rate, benefit, label = 'App benefit for ' + leg +' with ' +reconstr_type, \
                     color = colors[it], linewidth=2, ls = linest)  
            
            
        plt.plot(c_acceptance_rate[np.argmax(benefit)], np.max(benefit), color = colors[it], \
                     marker='*', markersize=10)
        max_val.append(np.max(benefit))


    plt.xlabel('Acceptance rate')
    plt.ylabel('Benefit')
    plt.title('Benefit curve')
    plt.legend(bbox_to_anchor=(1, 1));

    plt.grid()
    _ = plt.legend(loc= 0, prop= {'size': 16})
    
    print(reconstr_type, ': Max(benefit_'+cols_to_plot[0]+') -  Max(benefit_'+cols_to_plot[1]+') =', np.round(max_val[0] - max_val[1],2)) 
       

__Consider a binary classification model $X -> Prob$, e.g. credit scoring__:

#### Load Data

Here we deal with loan applications that we accepted or rejected previosuly based on some old decision policy. So, we load **accepted** and **rejected** samples. Accepted sample will be used to train ML model, and rejected will be used to adjust our trained model during reject inference procedure.

In [ ]:
df_train_rej = pd.read_csv('../data/hidden-errors/rejected_clients.csv')
df_train = pd.read_csv('../data/hidden-errors/accepted_clients.csv')

In [ ]:
df_train.head()

__Column description:__

- `issue_d` - The month which the loan was funded
- `addr_state` - The state provided by the borrower in the loan application
- `emp_title` - The job title supplied by the Borrower when applying for the loan.
- `installment` - The monthly payment owed by the borrower if the loan originates.
- `dti` - A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income.
- `funded_amnt` - The total amount committed to that loan at that point in time.
- `annual_inc` - The self-reported annual income provided by the borrower during registration.
- `emp_length` - Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years. 
- `term` - The number of payments on the loan. Values are in months and can be either 36 or 60.
- `inq_last_6mths` - The number of inquiries in past 6 months (excluding auto and mortgage inquiries)
- `mths_since_recent_inq` - Months since most recent inquiry.
- `delinq_2yrs` - The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years
- `chargeoff_within_12_mths` - Number of charge-offs within 12 months
- `num_accts_ever_120_pd` - Number of accounts ever 120 or more days past due
- `num_tl_90g_dpd_24m` - Number of accounts 90 or more days past due in last 24 months
- `acc_open_past_24mths` - Number of trades opened in past 24 months.
- `avg_cur_bal` - Average current balance of all accounts
- `tot_hi_cred_lim` - Total high credit/credit limit
- `delinq_amnt` - The past-due amount owed for the accounts on which the borrower is now delinquent.

And all categorical variables are encoded:
- `sub_grade` - External assigned loan subgrade
- `purpose` - A category provided by the borrower for the loan request.
- `home_ownership` - The home ownership status provided by the borrower during registration or obtained from the credit report. Our values are: RENT, OWN, MORTGAGE, OTHER


__Target variable:__
- `def`

#### Modelling
We can train our ML model, that will be implemented as a new decision policy, only on sample of accepted loans, because for that sample we know the target values (default or non-default events)

##### *1. Define target*

In [ ]:
targ_cols = [i for i in df_train.columns if i != 'def']

X_train = df_train[targ_cols]
y_train = df_train["def"]

X_train_rej = df_train_rej[targ_cols]
y_train_rej = df_train_rej["def"]

##### *2. Define FP and FN costs*

Financial result of model performance depends on FP and FN error

In [ ]:
S = 900 # amount of loan
r = 0.035 # interest rate
lgd = 0.25 # losses in case of default
e_fp = r * S # 1 type error cost
e_fn = lgd * S # 2 type error cost

Define short list of features for model

In [ ]:
short_list = ['installment', 'dti', 'funded_amnt', 'annual_inc', 'emp_length', 'term',
       'inq_last_6mths', 'mths_since_recent_inq', 'delinq_2yrs',
       'chargeoff_within_12_mths', 'num_accts_ever_120_pd',
       'num_tl_90g_dpd_24m', 'acc_open_past_24mths', 'avg_cur_bal',
       'tot_hi_cred_lim', 'delinq_amnt', 'purpose_car', 'purpose_credit_card',
       'purpose_debt_consolidation', 'purpose_home_improvement',
       'purpose_house', 'purpose_major_purchase', 'purpose_medical',
       'purpose_moving', 'purpose_other', 'purpose_renewable_energy',
       'purpose_small_business', 'purpose_vacation', 'purpose_wedding',
       'sub_grade_A1', 'sub_grade_A2', 'sub_grade_A3', 'sub_grade_A4',
       'sub_grade_A5', 'sub_grade_B1', 'sub_grade_B2', 'sub_grade_B3',
       'sub_grade_B4', 'sub_grade_B5', 'sub_grade_C1', 'sub_grade_C2',
       'sub_grade_C3', 'sub_grade_C4', 'sub_grade_C5', 'sub_grade_D1',
       'sub_grade_D2', 'sub_grade_D3', 'sub_grade_D4', 'sub_grade_D5',
       'sub_grade_E1', 'sub_grade_E2', 'sub_grade_E3', 'sub_grade_E4',
       'sub_grade_E5', 'sub_grade_F1', 'sub_grade_F2', 'sub_grade_F3',
       'sub_grade_F4', 'sub_grade_F5', 'sub_grade_G1', 'sub_grade_G2',
       'sub_grade_G3', 'sub_grade_G4', 'sub_grade_G5',
       'home_ownership_MORTGAGE', 'home_ownership_NONE',
       'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT']

len(short_list)

### Reject inference

__Reject inference steps:__
1. Train model
2. Score rejected clients
3. Simulate target event for rejected clients based on Prob at step 2
4. Retrain model on united dataset

Repeat 1-4 until models at Step 1 and Step 4 are close in terms of parameters value


In [ ]:
#define model architecture
arc = DecisionTreeClassifier(max_depth = 3, random_state= 123)
#Step1: train model
model_tree = arc.fit(X_train[short_list], y_train)

stop_crit = [0] * len(short_list)
while True:
    X_train_all = X_train[short_list]
    X_train_all['weight'] = 1
    y_train_all = pd.DataFrame(y_train)
    
    
    #Step2: score rejected clients
    weights_rej = model_tree.predict_proba(X_train_rej[short_list])[:,1]
    
    
    #Step3: simulate target event for rejected clients
        #duplicate the rows 
    X_train_rej_inf_0 = X_train_rej[short_list]        
    X_train_rej_inf_1 = X_train_rej[short_list]
        #assign each target equal 0 and 1
    y_train_rej_inf_0 = pd.DataFrame(np.zeros(shape=len(weights_rej)), columns= ['def'])
    y_train_rej_inf_1 = pd.DataFrame(np.ones(shape=len(weights_rej)), columns= ['def'])
        #assign weight equal to p_i and 1 - p_i for every duplicated row
    X_train_rej_inf_0['weight'] = 1 - weights_rej
    X_train_rej_inf_1['weight'] = weights_rej

    

    #Step4: retrain model on united dataset
    X_train_all = pd.concat((X_train_all, X_train_rej_inf_1, X_train_rej_inf_0))
    y_train_all = pd.concat((y_train_all, y_train_rej_inf_1, y_train_rej_inf_0))
    
    model_tree = arc.fit(X_train_all[short_list], y_train_all, sample_weight = list(X_train_all['weight']))
    
    #check that models are close in terms of parameters value
    if np.abs(stop_crit - model_tree.feature_importances_).all() <= 1e-4:
        break

    stop_crit = model_tree.feature_importances_



In [ ]:
#define df with estimations
X_train_rej['y_hat'] = model_tree.predict_proba(X_train_rej[short_list])[:,1]
X_train_rej['y'] = y_train_rej

X_train['y_hat'] = y_train
X_train['y'] = y_train

X_train_with_rej = X_train_rej
X_train_with_rej = pd.concat((X_train_with_rej, X_train))
X_train_with_rej['prob'] = model_tree.predict_proba(X_train_with_rej[short_list])[:,1]
X_train_with_rej.shape
X_train_with_rej_tree = X_train_with_rej.copy()

In [ ]:
print('ROC_AUC for model: ', roc_auc_score(X_train_with_rej_tree.y, X_train_with_rej_tree.prob))

In our study case we have keep true target values $y$ for both rejected and accepted clients which, of course, is not possible in real life. However, we do so in order to show how similar our approximation of benefit curve will be to the true benefit curve in case of large number of previously rejected clients. 

We also have the target _$y_{hat}$_ that we estimated.
$y_{hat} = y$ for clients with known target and $y_{hat} = y_{rej\_inf}$ for rejected clients.

Let's see how rejected inference works.


___
**Plot benefit curve** to see the dependence between the benefit and acceptance rate for our first ML model (decision tree)

After iterating through various threshold levels $a$ (and corresponding acceptance rates $c$), we can calculate cumulative expected benefit: $Benefit(c) = \sum_{i:Prob_i \le a} (1-Y_i) \bullet e_{FP} - Y_i \bullet e_{FN}$

As $Y_i$ is probabilistic for rejected clients all formulas give us expected result.


In [ ]:
thr = np.linspace(0,1,41) #41

In [ ]:
benefit_plot(X_train_with_rej_tree, leg = 'Tree model')

Let us now try to build more complex ML model, for instance with the help of gradient boosting classifier. We expect it to be more accurate and, therefore, bring more benefit given fixed acceptance rate.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
#define model architecture
arc = GradientBoostingClassifier(learning_rate=0.2, random_state= 123)
#Step1: train model
model_boosting = arc.fit(X_train[short_list], y_train)

stop_crit = [0] * len(short_list)
while True:
    X_train_all = X_train[short_list]
    X_train_all['weight'] = 1
    y_train_all = pd.DataFrame(y_train)
    
    
    #Step2: score rejected clients    
    weights_rej = model_boosting.predict_proba(X_train_rej[short_list])[:,1]

    
    #Step3: simulate target event for rejected clients
        #duplicate the rows 
    X_train_rej_inf_0 = X_train_rej[short_list]
    X_train_rej_inf_1 = X_train_rej[short_list]
        #assign each target equal 0 and 1
    y_train_rej_inf_0 = pd.DataFrame(np.zeros(shape=len(weights_rej)), columns= ['def'])
    y_train_rej_inf_1 = pd.DataFrame(np.ones(shape=len(weights_rej)), columns= ['def'])
        #assign weight equal to p_i and 1 - p_i for every duplicated row    
    X_train_rej_inf_1['weight'] = weights_rej
    X_train_rej_inf_0['weight'] = 1 - weights_rej

    
    #Step4: retrain model on united dataset
    X_train_all = pd.concat((X_train_all, X_train_rej_inf_1, X_train_rej_inf_0))
    y_train_all = pd.concat((y_train_all, y_train_rej_inf_1, y_train_rej_inf_0))

    model_boosting = arc.fit(X_train_all[short_list], y_train_all, sample_weight = list(X_train_all['weight']))
    
    #check that models are close in terms of parameters value
    if np.abs(stop_crit - model_boosting.feature_importances_).all() <= 1e-4:
        break

    stop_crit = model_boosting.feature_importances_


In [ ]:
#define df with estimations
X_train_rej['y_hat'] = model_boosting.predict_proba(X_train_rej[short_list])[:,1]
X_train['y_hat'] = y_train

X_train_rej['y'] = y_train_rej
X_train['y'] = y_train

X_train_with_rej = X_train_rej
X_train_with_rej = pd.concat((X_train_with_rej, X_train))
X_train_with_rej['prob'] = model_boosting.predict_proba(X_train_with_rej[short_list])[:,1]
X_train_with_rej.shape
X_train_with_rej_boost = X_train_with_rej.copy()

In [ ]:
print('ROC_AUC for model: ', roc_auc_score(X_train_with_rej_boost.y, X_train_with_rej_boost.prob))

___
**Plot benefit curve** 

Here we use above-defined function benefit_plot(). Look it up to make sure you remember how it works.

In [ ]:
benefit_plot(X_train_with_rej_boost, leg = 'Boosting model')

Plot benefit curve for two models to compare them

In [ ]:
plt.figure(figsize=(12,8))

benefit_plot(X_train_with_rej_boost, leg='Boosting model')
benefit_plot(X_train_with_rej_tree, leg='Tree model', linest='--')
plt.grid();

___

There are some tasks to the examples above.

### Task 1 
#### __Rewrite the__ `benefit_plot` to `benefit_calc` __function and calculate the difference between the max benefit for approximate and true curve for both tree and boosting models_

In [ ]:
# your code here

def benefit_calc_true(X_train):
    
    dict_benefit = {}
    for it, i in enumerate(['y','y_hat']):

        benefit = []    
        c_acceptance_rate = []
        
        y_hat = X_train[i]
        y_ar = X_train['prob']
        
        for t in thr:            
            #calculate accaptance rate as amount of non-defaulted clients
            c_acceptance_rate.append((len(y_ar) - np.sum((y_ar > t)*1.)) / len(y_ar))
            curr_y_hat = y_hat[X_train['prob'] <= t]    
            #calculate the financial effect
            benefit.append(np.sum((1 - curr_y_hat)* e_fp - curr_y_hat * e_fn))
                        
        
        # YOUR CODE 
        dict_benefit[i] = pd.DataFrame(columns= ['acc_rate', 'benefit'])
        dict_benefit[i]['acc_rate'] = c_acceptance_rate 
        dict_benefit[i]['benefit'] = benefit 
        ######
    
    return dict_benefit         
        

dict_benefit_tree = benefit_calc_true(X_train_with_rej_tree)
dict_benefit_boost = benefit_calc_true(X_train_with_rej_boost)

max_benefit_y_tree = max(dict_benefit_tree['y']['benefit'])
max_benefit_y_hat_tree = max(dict_benefit_tree['y_hat']['benefit'])

max_benefit_y_boosting = max(dict_benefit_boost['y']['benefit'])
max_benefit_y_hat_boosting = max(dict_benefit_boost['y_hat']['benefit'])

print("Max benefit y tree: {:.0f}".format(max_benefit_y_tree))
print("Max benefit y hat tree: {:.0f}".format(max_benefit_y_hat_tree))
print("Max benefit y boosting: {:.0f}".format(max_benefit_y_boosting))
print("Max benefit y hat boosting: {:.0f}".format(max_benefit_y_hat_boosting))

tree_diff = np.abs(max_benefit_y_tree - max_benefit_y_hat_tree)
boosting_diff = np.abs(max_benefit_y_boosting - max_benefit_y_hat_boosting)

print("Boosting diff: {:.2f}".format(boosting_diff))
print("Tree diff: {:.2f}".format(tree_diff))

# your code here


### Task 2 
#### __Use the function above to calculate the difference between estimated benefit for tree and boosting models at $acceptance\_rate = 0.4$__


In [ ]:
#Use approximation function
def get_benefit_at_point(a_r, df):
    x1 = df[(df['acc_rate'] <= 0.4)]['acc_rate'].iloc[-1]
    x2 = df[(df['acc_rate'] > 0.4)]['acc_rate'].iloc[0]
    y1 = df[(df['acc_rate'] <= 0.4)]['benefit'].iloc[-1]
    y2 = df[(df['acc_rate'] > 0.4)]['benefit'].iloc[0]
    return (y2 - y1)/ (x2 - x1) * a_r + y1 - (y2 - y1)/ (x2 - x1) * x1

In [ ]:
# your code here
diff_models_benefit_at_point = (
    get_benefit_at_point(a_r=0.4, df=dict_benefit_tree['y_hat']) - 
    get_benefit_at_point(a_r=0.4, df=dict_benefit_boost['y_hat'])
)

print("Diff estimated benefit boost - tree: {:.0f}".format(diff_models_benefit_at_point))

# your code here


### Metalearning

At this moment, we have plots of approximated benefits for two models which are built independently based on its own predictions. Again, keep in mind that in real life we can observe only blue benefit curves as soon as we do not know true target values for rejected clients by previously settled decision policy.

Next, we want to compare approximated benefit curves built using one single target proxy. So that, differences in benefit curves are explained only by model quality itself and not just by different reject inferences. We will use metalearning to calculate unified target proxy. 

So let us put our data together.

In [ ]:
X_train_with_rej_tree.head()

In [ ]:
X_train_with_rej_boost.head()

In [ ]:
#define df with estimations from both models
X_train_meta = X_train_with_rej[short_list]
X_train_meta['y'] = X_train_with_rej['y']

X_train_meta['y_hat_tree'] = X_train_with_rej_tree['y_hat']
X_train_meta['prob_tree'] = X_train_with_rej_tree['prob']

X_train_meta['y_hat_boost'] = X_train_with_rej_boost['y_hat']
X_train_meta['prob_boost'] = X_train_with_rej_boost['prob']

In [ ]:
X_train_meta.tail()

In [ ]:
X_train_meta.head()

In [ ]:
#define flag of rejection for cliens
X_train_meta['rej_flag'] = X_train_meta.y_hat_tree.apply(lambda x: False if x==1 or x==0 else True)
X_train_meta_train = X_train_meta[X_train_meta['rej_flag'] == False]
X_train_meta_pred = X_train_meta[X_train_meta['rej_flag'] == True]

_Define meta data:_
* $𝐹𝑙𝑎𝑔=1$ if boosting prediction is closer to correct answer than tree prediction
* $𝐹𝑙𝑎𝑔=0$, otherwise

In [ ]:
X_train_meta_train['flag'] = (np.abs(X_train_meta_train.prob_tree - X_train_meta_train.y) >= np.abs(X_train_meta_train.prob_boost - X_train_meta_train.y))

So for each client with known target value we labeled which model (decision tree or gradient boosting classifier) is more accurate with its prediction. 

We will train an auxiliary classifier $P\_aux$ which will predict which model is more accurate in its prediciton for particular client with given features. 

Prediction $P\_aux$ is in fact a probability that boosting is likely to be more accurate than decision tree.

Finally, we use it as a weight to calculate unified target proxy for clients with unknown target event.

In [ ]:
model_tree_meta = DecisionTreeClassifier(random_state=0, max_depth = 6).fit(X_train_meta_train[short_list], X_train_meta_train['flag'])

X_train_meta_pred['flag'] = model_tree_meta.predict_proba(X_train_meta_pred[short_list])[:,1]

X_train_meta_pred['blend_y'] = (1 - X_train_meta_pred['flag']) * X_train_meta_pred.y_hat_tree +\
                                X_train_meta_pred['flag']*X_train_meta_pred.y_hat_boost
X_train_meta_train['blend_y'] = X_train_meta_train['y']

X_train_meta['y_hat'] = pd.concat((X_train_meta_pred['blend_y'], X_train_meta_train['blend_y']))



In [ ]:
X_train_meta.head()

Plot all benefit curves with reconstructed target event

In [ ]:
plt.figure(figsize=(12,8))

benefit_plot(X_train_with_rej_boost, leg='Boosting model')
benefit_plot(X_train_meta, leg='Boosting model', prob_col='prob_boost',  plot_y_true=False, linest='-.')


benefit_plot(X_train_with_rej_tree, leg='Tree model', linest=':')
benefit_plot(X_train_meta, leg='Tree model', linest='--', prob_col='prob_tree', plot_y_true=False)

plt.grid();

In our example we get that the higher model quality metrics are, the higher is financial result (in general)

### Task 3
#### __Calculate the average weight of the tree and boosting classifiers on known y. Draw conclusions__

In [ ]:
# your code here
mean_tree = 1 - X_train_meta_train['flag'].mean()
mean_boost = X_train_meta_train['flag'].mean()

# your code here
